# CSE291 HW7

(a)

To describe how I got my embeddings, I will add descpritions for all cells and how they come to the final embeddings.

Firstly, download the nltk package as I need

In [1]:
pip install nltk

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/Note: you may need to restart the kernel to use updated packages.



In [2]:
import nltk #import nltk

I tried download brown directly, but it failed, here's how I got the path of nltk and after that I downloaded manually.

In [3]:
nltk.data.path

['C:\\Users\\Lenovo/nltk_data',
 'F:\\anaconda\\anaconda2\\nltk_data',
 'F:\\anaconda\\anaconda2\\share\\nltk_data',
 'F:\\anaconda\\anaconda2\\lib\\nltk_data',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data',
 'C:\\nltk_data',
 'D:\\nltk_data',
 'E:\\nltk_data']

after downloading, I import it and check it

In [4]:
from nltk.corpus import brown

words = brown.words()

In [5]:
len(words)

1161192

In [6]:
type(words)

nltk.corpus.reader.util.ConcatenatedCorpusView

Get a string of brown words for later use.

In [7]:
Words=' '.join(words)

Import all the packages I might use

In [8]:
import string
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk import punkt
import math
import numpy as np
import pandas as pd
import random
import sklearn
import scipy
from pylab import rcParams
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

For this cell, I use two funtions. First one is for processing, which removes punctuation and stopwords and those non-alphabet words I think might not help for embeddings. Second one is for counting the frequency for each word in processed text streams. By using these two functions, get data cleaned.

In [9]:
#the methods for processing my data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("","", string.punctuation))
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Remove non-alphabetic characters using regex
    words = [re.sub('[^a-zA-Z]', '', word) for word in words if word]
    
    return words

def count_word_frequency(words):
    # Count word frequencies
    word_counts = Counter(words)
    
    return word_counts

# Example usage

words = preprocess_text(Words)
words=[word for word in words if word!='']
word_frequencies = count_word_frequency(words)

This cell, is get the whole list sorted in the order of frequency, the first 10 is shown.

In [10]:
def comp(x):
    return x[1]


theWholeList=[(i,j) for i,j in word_frequencies.items()]
theWholeList.sort(key=comp,reverse=True)
theWholeList[:10]

[('one', 3297),
 ('would', 2714),
 ('said', 1961),
 ('new', 1635),
 ('could', 1601),
 ('time', 1598),
 ('two', 1412),
 ('may', 1402),
 ('first', 1361),
 ('like', 1292)]

For what I've done now, I got the whole list for words right now. Using that list, below is how I got short list C and vocabulary V by selecting first 100 (since 100 dimensional for later embeddings) and first 5000. Since the list is in order of frequency, this way, fits the requirement.

In [11]:
C={word:num for word,num in theWholeList[:100]}   #A shorter list C which we shall call context words.
V={word:num for word,num in theWholeList[:5000]}    #A vocabulary V , consisting of a few thousand of the most commonly-occurringwords.

For this cell, is doing window conut. That is to say, like mentioned in the question,  get four words around the word in vocabulary which are two before the word w and two after and all of them in short list C.

In [12]:
context_word_counts = {word: Counter() for word in V}
window_size = 2

for i, word in enumerate(words):
    if word in V:
        start = max(0, i - window_size)
        end = min(i + window_size + 1, len(words))
        context = [words[j] for j in range(start, end) if (i!=j and words[j] in C)]
        #print(context)
        context_word_counts[word].update(context)


here is one example.

In [13]:
context_word_counts['one']

Counter({'make': 31,
         'first': 36,
         'two': 86,
         'three': 24,
         'years': 23,
         'would': 77,
         'time': 66,
         'united': 4,
         'men': 16,
         'said': 54,
         'well': 25,
         'number': 13,
         'get': 22,
         'people': 24,
         'states': 3,
         'go': 16,
         'since': 10,
         'may': 49,
         'another': 108,
         'one': 84,
         'last': 27,
         'every': 19,
         'take': 24,
         'went': 11,
         'school': 11,
         'made': 25,
         'never': 28,
         'year': 38,
         'new': 25,
         'good': 37,
         'day': 60,
         'used': 9,
         'way': 45,
         'small': 14,
         'come': 14,
         'see': 17,
         'also': 21,
         'found': 18,
         'left': 19,
         'world': 13,
         'must': 40,
         'house': 16,
         'without': 13,
         'head': 13,
         'use': 14,
         'back': 19,
         'away': 9,
 

This part is computing probablity. Using 'context_word_counts', we can get conditional probablity by computing their appreance with each word w over all words aperance around this specific word w. And computing Pr(c) by simply counting its apperance in all words.

In [14]:
#compute Pr(c|w)  
conditional_probabilities = {}

for word in V:
    context_counts = context_word_counts[word]  

    total_count = sum(context_counts.values())  

    
    conditional_probabilities[word] = {}
    for context_word in context_counts:
        prob = context_counts[context_word] / total_count
        conditional_probabilities[word][context_word] = prob
        
#--------------------------------------------------------------------------------------------------------------------        
# compute Pr(c)
context_word_total_counts = Counter()

for word in V:
    context_word_total_counts.update(context_word_counts[word])

Prc = {}

total_count = sum(context_word_total_counts.values())

for context_word in C:
    prob = context_word_total_counts[context_word] / total_count
    Prc[context_word] = prob

In [15]:
conditional_probabilities['one']

{'make': 0.013796172674677348,
 'first': 0.01602136181575434,
 'two': 0.03827325322652426,
 'three': 0.010680907877169559,
 'years': 0.010235870048954161,
 'would': 0.03426791277258567,
 'time': 0.029372496662216287,
 'united': 0.0017801513128615932,
 'men': 0.007120605251446373,
 'said': 0.02403204272363151,
 'well': 0.011125945705384957,
 'number': 0.005785491766800178,
 'get': 0.009790832220738763,
 'people': 0.010680907877169559,
 'states': 0.0013351134846461949,
 'go': 0.007120605251446373,
 'since': 0.004450378282153983,
 'may': 0.021806853582554516,
 'another': 0.04806408544726302,
 'one': 0.037383177570093455,
 'last': 0.012016021361815754,
 'every': 0.008455718736092568,
 'take': 0.010680907877169559,
 'went': 0.004895416110369382,
 'school': 0.004895416110369382,
 'made': 0.011125945705384957,
 'never': 0.012461059190031152,
 'year': 0.016911437472185136,
 'new': 0.011125945705384957,
 'good': 0.016466399643969738,
 'day': 0.0267022696929239,
 'used': 0.004005340453938585,
 '

In [16]:
Prc['one']

0.04286076484259455

Now, with all the distributions, we compute the embeddings as what is instrcuted by choosing max number of 0 and log(Pr(c|w)/Pr(c)) for each entry.

In [17]:
word_embeddings = {word: [0] * len(C) for word in V}

for word in V:
    for i, context_word in enumerate(C):
        if context_word== word:continue
        #print(word+','+context_word)
        prob_c_w = conditional_probabilities[word][context_word] if context_word in conditional_probabilities[word] else 1e-9
        prob_c = Prc[context_word] if Prc[context_word]!=0 else 1e-9
        word_embeddings[word][i] = max(0, math.log(prob_c_w / prob_c))

Above is all the steps of how I got my embeddings.

(b)

In [18]:
def cosine_similarity(embedding_w, embedding_w0):
    dot_product = np.dot(embedding_w, embedding_w0)
    norm_w = np.linalg.norm(embedding_w)
    norm_w0 = np.linalg.norm(embedding_w0)
    cos_sim = dot_product / (norm_w * norm_w0)
    return cos_sim

def pickWords(theWholeList):
    random_numbers = random.sample([i for i in range(5000)], 25)
    wordlist=[theWholeList[num][0] for num in random_numbers]
    return wordlist

wordlist=pickWords(theWholeList)
wordpair=[]
for word in wordlist:
    tmp=None
    sim=-1
    for cmp_word in V:
        if cmp_word==word:continue
        new_sim=cosine_similarity(word_embeddings[word], word_embeddings[cmp_word])
        if new_sim>sim:
            sim=new_sim
            tmp=cmp_word
    wordpair.append((word,tmp,sim))


<ipython-input-18-99f9bd50444b>:5: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = dot_product / (norm_w * norm_w0)


In [19]:
wordpair

[('net', 'published', 0.5283024425195768),
 ('stress', 'food', 0.5921562496091787),
 ('rehabilitation', 'earliest', 0.5686828458817361),
 ('luxury', 'strategic', 0.595100590454909),
 ('moral', 'meaning', 0.5768974326809412),
 ('accurately', 'carry', 0.49920045735376894),
 ('meant', 'heard', 0.6154985035173524),
 ('listeners', 'passages', 0.5680268439666191),
 ('running', 'protest', 0.6102340621641896),
 ('equal', 'increased', 0.5634571984172476),
 ('bobby', 'midnight', 0.5617157954180494),
 ('deegan', 'hit', 0.5680495091957429),
 ('worlds', 'source', 0.49370578763244316),
 ('initial', 'food', 0.5154134586443299),
 ('oral', 'obviously', 0.5693072506462691),
 ('surrounded', 'speaking', 0.5079587285178777),
 ('merely', 'involved', 0.6168889069028188),
 ('lacked', 'favorable', 0.5913706468340824),
 ('light', 'solution', 0.5903190171148776),
 ('shoulder', 'pressed', 0.662694812749987),
 ('possibilities', 'ordinary', 0.5845108887687327),
 ('receive', 'establish', 0.5844267530708848),
 ('feel

Well, for this result, from my perspectives, I have to some make sense, some not. For instance, some you can tell the link, like authors and critics, because some author write critical things in their work; sit and station, one can sit at the bus station waiting for his or her bus. But some are meaningless, like oral and obviously, lacked and favorable, some I cannot see the strong relations.

(c)

In [20]:
embeddings=[word_embeddings[word] for word in V]
cosine_similarities = pairwise_distances(embeddings, metric='cosine')
kmeans = KMeans(n_clusters=100, random_state=0)
clusters = kmeans.fit_predict(cosine_similarities)

In [21]:
kmeans.labels_

array([61, 86, 86, ..., 17, 99, 10])

In [22]:
cluster_dict = {i: [] for i in range(100)}
V_list=[word for word in V]
for i, label in enumerate(kmeans.labels_):
    cluster_dict[label].append(V_list[i])

In [23]:
cluster_dict

{0: ['via',
  'climbed',
  'track',
  'knees',
  'edges',
  'dressed',
  'johnson',
  'keys',
  'rector',
  'trail',
  'ceiling',
  'fred',
  'concepts',
  'owen',
  'aboard',
  'wally',
  'pile',
  'pulling',
  'glanced',
  'fist',
  'slid',
  'weekly',
  'pete',
  'dressing',
  'yelled',
  'russ',
  'skirt',
  'breathing',
  'hoag',
  'loud',
  'brushed',
  'peered',
  'cottage',
  'twisted',
  'straightened'],
 1: ['thus',
  'line',
  'c',
  'rate',
  'b',
  'lines',
  'minutes',
  'space',
  'range',
  'p',
  'temperature',
  'function',
  'mass',
  'r',
  'operating',
  'inches',
  'address',
  'laid',
  'column',
  'sections',
  'cutting',
  'measured',
  'resolution',
  'sets',
  'associated',
  'drop',
  'solution',
  'neighborhood',
  'hence',
  'integration',
  'load',
  'volumes',
  'separated',
  'exists',
  'comparable',
  'assumption',
  'match',
  'meets',
  'findings',
  'structures',
  'accordingly',
  'compare',
  'managers',
  'votes'],
 2: ['england',
  'dramatic',


In [24]:
cluster_dict[5]

['however',
 'number',
 'present',
 'problem',
 'forces',
 'stock',
 'ideas',
 'series',
 'theory',
 'somewhat',
 'lower',
 'described',
 'image',
 'activity',
 'justice',
 'staff',
 'pattern',
 'principle',
 'becomes',
 'success',
 'patient',
 'collection',
 'crisis',
 'youth',
 'presented',
 'file',
 'hearing',
 'region',
 'brief',
 'phase',
 'unity',
 'independence',
 'gross',
 'capable',
 'headquarters',
 'site',
 'european',
 'tension',
 'sensitive',
 'requires',
 'views',
 'protestant',
 'choose',
 'discovery',
 'argue']

In [25]:
cluster_dict[8]

['way',
 'get',
 'got',
 'enough',
 'better',
 'told',
 'going',
 'look',
 'asked',
 'money',
 'keep',
 'job',
 'wife',
 'wanted',
 'girl',
 'mother',
 'leave',
 'hard',
 'idea',
 'father',
 'couldnt',
 'tried',
 'getting',
 'police',
 'talk',
 'ready',
 'anyone',
 'chance',
 'husband',
 'lot',
 'stay',
 'waiting',
 'wont',
 'pretty',
 'bit',
 'walk',
 'talking',
 'hadnt',
 'fight',
 'happy',
 'hed',
 'giving',
 'wait',
 'someone',
 'shed',
 'nice',
 'watching',
 'knife',
 'send',
 'jones',
 'mercer',
 'smiled',
 'asking',
 'kill',
 'please',
 'talked',
 'minute',
 'wished',
 'breakfast',
 'anyway',
 'fool',
 'drunk']

I think some are coherent to some extent. There are some potential meanings within them, above is some examples.